## Setups

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import pybroker as pyb
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance, StrategyConfig

from datetime import datetime, timedelta
from numba import njit
import numpy as np

In [2]:
config = StrategyConfig(max_long_positions=5)
pyb.param('target_size', 1 / config.max_long_positions)
pyb.param('rank_threshold', 6)
pyb.enable_data_source_cache('yfinance')



## Indicators

In [3]:
import talib as ta

def ew13612_fnc(data):
    roc12=ta.ROC(data.adj_close, timeperiod=12)
    roc6=ta.ROC(data.adj_close, timeperiod=6)
    roc3=ta.ROC(data.adj_close, timeperiod=3)
    roc1=ta.ROC(data.adj_close, timeperiod=1)
    
    return (roc12 + roc6 + roc3 + roc1)/4

ew13612_idx = pyb.indicator('ew13612_idx', lambda data: ew13612_fnc(data))

def adj_close_fnc(data):
    v_adj_close=data.adj_close
    
    return v_adj_close

adj_close_idx = pyb.indicator('adj_close_idx', lambda data: adj_close_fnc(data))


def pctl_chnl_fnc(data, lookback):

    # @njit  # Enable Numba JIT.
    def vec_pctl_chnl_fnc(data):
        rolling_close = data.rolling(lookback)
        pctl25 = rolling_close.quantile(0.25)
        pctl75 = rolling_close.quantile(0.75)
        closing_price = data.adj_close

        result = np.where(closing_price < pctl25, -1, np.where(closing_price > pctl75, 1, 0))
        return result
    return vec_pctl_chnl_fnc(data)
        
pc252_idx = pyb.indicator('pc252_idx', pctl_chnl_fnc, lookback=200)



## Data

In [ ]:
pctChannelPosition <- function(prices,
                               dayLookback = 60,
                               lowerPct = .25, upperPct = .75) {
  # Create a matrix of leading NAs to ensure proper alignment
  leadingNAs <- matrix(nrow = dayLookback - 1, ncol = ncol(prices), NA)

  # Calculate upper channel values
  upperChannels <- runquantile(prices, k = dayLookback, probs = upperPct, endrule = "trim")
  upperQ <- xts(rbind(leadingNAs, upperChannels), order.by = index(prices))

  # Calculate lower channel values
  lowerChannels <- runquantile(prices, k = dayLookback, probs = lowerPct, endrule = "trim")
  lowerQ <- xts(rbind(leadingNAs, lowerChannels), order.by = index(prices))

  # Initialize a matrix for positions
   positions <- xts(matrix(nrow = nrow(prices), ncol = ncol(prices), NA), order.by = index(prices))

  # Set positions based on the channel crossover conditions
  positions[prices > upperQ & lag(prices) < upperQ] <- 1    # Cross up
  positions[prices < lowerQ & lag(prices) > lowerQ] <- -1   # Cross down

  # Fill missing values using last observation carried forward (na.locf)
  positions <- na.locf(positions)

  # Set remaining NA positions to 0
  positions[is.na(positions)] <- 0

  # Set column names of positions to match price data columns
  colnames(positions) <- colnames(prices)

  return(positions)
}

In [8]:
yfinance = YFinance()
symbols=['SPY','PDBC','LQD','VNQ']
df = yfinance.query(symbols, start_date='1/1/2015', end_date='8/1/2023')

rolling_close = df['adj_close'].rolling(20)

dir(rolling_close)

# Group the DataFrame by the 'group' column
# grouped = df.rolling(20'group')
# dir(grouped)
# # Output data with space between groups
# for name, group in grouped:
#     print(f"Group {name}:")
#     print(group)
#     print()

Loaded cached bar data.



['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_blockwise',
 '_apply_pairwise',
 '_apply_series',
 '_apply_tablewise',
 '_attributes',
 '_check_window_bounds',
 '_create_data',
 '_dir_additions',
 '_generate_cython_apply_func',
 '_get_window_indexer',
 '_gotitem',
 '_index_array',
 '_insert_on_column',
 '_internal_names',
 '_internal_names_set',
 '_is_protocol',
 '_make_numeric_only',
 '_numba_apply',
 '_obj_with_exclusions',
 '_on',
 '_prep_values',
 '_raise_monotonic_error',
 '_resolve_output',
 '_selected_obj',

In [ ]:


md = pd.read_csv('D:/Documents/Development/Jupyter/pybroker/market_dates.csv', parse_dates=['trade_dt'], index_col='trade_dt')
# md.sort_values(by=['trade_dt'])[-20:]

yfinance = YFinance()

# symbols=['SPY','QQQ','IWM','VGK','EWJ','EEM','VNQ','PDBC','GLD','HYG','LQD','TLT']
symbols=['SPY','PDBC','LQD','VNQ']

df = yfinance.query(symbols, start_date='1/1/2015', end_date='8/1/2023')

# df.rename(columns={"date":"trade_dt"}, inplace=True)
# df.set_index('trade_dt', inplace=True)
#[['Symbol']=='EEM']
dom_df = pd.merge(df, md[['month_last_day']], left_on='date', right_on='trade_dt')
eom_df = dom_df[dom_df['month_last_day'] == 'Y'][['date','symbol','adj_close']]
eom_df

## Percentile Score

In [ ]:
pc_score = eom_df.copy()
for lookback in [3,6,9,12]:
    rolling_close = pc_score['adj_close'].rolling(lookback)
    pctl25 = rolling_close.quantile(0.25)
    pctl75 = rolling_close.quantile(0.75)
    close_price = pc_score['adj_close']

    # result = pd.Series(-1, index=close_price.index)
    result = pd.Series(-1, index=close_price.index)
    last_result=-1
    for idx, value in close_price.items():
        if close_price[idx] > pctl75[idx]:
            result[idx]=1
            last_result=1
        elif (close_price[idx] > pctl25[idx]) & (last_result==1):
            result[idx]=1
            last_result=1
        
        pc_column = f"pc_{lookback}"
        pc_score[pc_column] = result

pc_score.set_index('date', inplace=True)
pc_score

## Channel Score

In [ ]:
pc_chan_score = pc_score.copy()
def sum_pc(row):
    pc_columns = ['pc_3', 'pc_6', 'pc_9', 'pc_12']
    return row[list(pc_columns)].sum()/4

pc_chan_score['pc_chan_score'] = pc_chan_score.apply(sum_pc, axis=1)
pc_chan_score['pc_chan_score_abs'] = pc_chan_score.apply(sum_pc, axis=1).abs()
pc_chan_score[-20:]

## Volatility

In [ ]:
pc_vol = df[['date','symbol','adj_close']].copy() # yfinance.query(symbols, start_date='1/1/2015', end_date='8/1/2023')
# pc_vol=pc_vol[['date','symbol','adj_close']].copy()

pc_vol['date'] = pd.to_datetime(pc_vol['date'])

# Sort the DataFrame by 'symbol' and 'date' to ensure proper order for rolling calculation
pc_vol.sort_values(by=['symbol', 'date'], inplace=True)

# Set 'date' as the index for the DataFrame
pc_vol.set_index('date', inplace=True)

# Calculate the rolling 20-day volatility for each symbol
pc_vol['vol20'] = pc_vol.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std())
# pc_vol['vol20_abs'] = pc_vol.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std()).abs()

# pc_vol[-60:].sort_values(by=['date','symbol'])
# pc_vol.groupby('symbol').size()/252
pc_vol.sort_values(by=['date', 'adj_close'])

##  Channel Score Plus Volatility 

In [ ]:
pc_vol.reset_index(inplace=True)

# Merge based on 'date' and 'symbol'
pc_chan_score_vol = pd.merge(
    pc_chan_score,
    pc_vol[['date','symbol','vol20']],
    on=['date', 'symbol'],
    how='left'
)

pc_chan_score_vol

## Volatility Adjusted Channel Score

In [ ]:
pc_vol.reset_index(inplace=True)

# Merge based on 'date' and 'symbol'
pc_chan_score_vol = pd.merge(
    pc_chan_score,
    pc_vol[['date','symbol','vol20']],
    on=['date', 'symbol'],
    how='left'
)

pc_vol_adj_score=pc_chan_score_vol.copy()
pc_vol_adj_score['chan_score_inv_vol'] = pc_chan_score_vol['pc_chan_score'] * (1 / pc_chan_score_vol['vol20'])
pc_vol_adj_score['chan_score_inv_vol_abs'] = pc_vol_adj_score['chan_score_inv_vol'].abs()
pc_vol_adj_score

## Weightings

In [ ]:
vol_adj_date_sum = pc_vol_adj_score.groupby('date')['chan_score_inv_vol_abs'].sum()
pc_score_vol_adj_dt_wght = pc_vol_adj_score.copy()

for date, value in vol_adj_date_sum.items():
    pc_score_vol_adj_dt_wght.loc[pc_score_vol_adj_dt_wght['date'] == date, 'vol_adj_sum_date'] = value

pc_score_vol_adj_dt_wght

In [ ]:

pc_pct_alloc=pc_score_vol_adj_dt_wght.copy()
# pc_pct_alloc['pct_alloc'] = pc_pct_alloc['pc_score_vol_adj'] / pc_pct_alloc['score_vol_adj_date_wght']
pc_pct_alloc['pct_alloc'] = (pc_pct_alloc['chan_score_inv_vol_abs'] / pc_pct_alloc['vol_adj_sum_date']).apply(lambda x: '{:.2f}%'.format(x * 100))


pc_pct_alloc[-20:]